#Introdução

Este projeto é uma simulação de entrevista em Data Science para a criação de um modelo de ML para prever fraudes.

O objetivo do projeto é criar um modelo que poderá prever quando um sistema poderá ou não reconhecer uma fraude.

O dataset já está tratado e otimizado e possui uma variável alvo 'fraud', que será usada para entender, através de regressão logística, como uma fraude acontece ou não.

In [1]:
import pandas as pd

## Subir a base e verificar se existem dados nulos

In [2]:
df = pd.read_csv('fraud_dataset.csv')
df.head()

,distance_from_home,distance_from_last_transaction,ratio_to_median_purchase_price,repeat_retailer,used_chip,used_pin_number,online_order,fraud
0,57.877857,0.311140,1.945940,1.0,1.0,0.0,0.0,0.0
1,10.829943,0.175592,1.294219,1.0,0.0,0.0,0.0,0.0
2,5.091079,0.805153,0.427715,1.0,0.0,0.0,1.0,0.0
3,2.247564,5.600044,0.362663,1.0,1.0,0.0,1.0,0.0
4,44.190936,0.566486,2.222767,1.0,1.0,0.0,1.0,0.0


## Variável Alvo: **fraud**

aplicar a verificação de valores nulos para entender se há valores ausentes

In [4]:
df.isnull().sum()

,0
distance_from_home,0
distance_from_last_transaction,0
ratio_to_median_purchase_price,0
repeat_retailer,0
used_chip,0
used_pin_number,0
online_order,0
fraud,0


aplicar uma verificação de desequilibrio com value counts, multiplicado por 100 e dividido em duas casas decimais.

In [6]:
round(df['fraud'].value_counts(normalize=True)*100,2)

,proportion
fraud,
0.0,91.26
1.0,8.74


Antes de aplicar a normalização ou padronização deveremos separar os dados em treino e teste para não haver data leakage para que meus dados sejam padronizados corretamente e não causem.

padronização na base inteira - data leakage (vazamento de dados)
ideia do teste é representar um dado que eu nunca vi antes, pode vazar estatisticas

## Holdout

Separação da base em treino e teste- técnica de hold out (80% treino e 20% teste)

In [8]:
from sklearn.model_selection import train_test_split

x = df.drop('fraud', axis=1)
y = df['fraud']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, stratify=y, random_state=7)

separadas as bases da variável target, agora nós usamos o shape para verificar o tamanho da base, se corresponde a divisão correta de treino e teste.

In [9]:
print(x_train.shape, x_test.shape)


(800000, 7) (200000, 7)


se eu tivesse variáveis de texto, teria que aplicar o label enconding, mas como não tem

usar o standard scaler para aplicar o z - score
verificar o dado em x = media/desvio padrão
criando o escalonamento para manter a amplitude dos dados, quando tem outliers
min/max scaler ( mudar nome da variavel para 'escalonado')
detalhe: usar o FIT para não deixar meus dados enviesados.

In [11]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

scaler = scaler.fit(x_train)

x_train_escalonado = scaler.transform(x_train)
x_test_escalonado = scaler.transform(x_test)

Pegar a menor classe com menor representação e dar um pump nela com oversampling

não esquecer de criar uma nova variável com a base de dados tunada com a questão de mesmo equilibrio para ambdas as classes

In [12]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(sampling_strategy='minority')

x_train_resampled, y_train_resampled = smote.fit_resample(x_train_escalonado, y_train)

#estratégia para aplicar o oversampling

In [14]:
print(pd.Series(y_train_resampled).value_counts(normalize=True)*100)

fraud
0.0    50.0
1.0    50.0
Name: proportion, dtype: float64


# REGRESSÃO LOGÍSTICA + VALIDAÇÃO CRUZADA

In [17]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

modelo = LogisticRegression(penalty='l2')

#validação cruzada, importante separar os folds, e depois criar o scores para as bases já otimizadas, mesmo tamanho de classe

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=7)
# calculo da media - usando os resammpled pq ja estao no mesmo tamanho de classe
scores = cross_val_score(modelo, x_train_resampled, y_train_resampled, cv=cv, scoring='roc_auc')
#


## Métrica usada: ROC AUC

o modelo me deu um métrica em ROC AUC  de 0.97

In [18]:
print(scores.mean())

0.9792708914086447


usamos dentro da y_pred os dados do x text escalonado pq o x test escalonado é a variavel que gera as previsões da variável y.

estamos na verdade colocando os dados de entrada, explicativos features, para gerar as previsões de y.

como nao sabemos o valor de x, y terá uma previsão com base no que foi treinado em x.

In [20]:

modelo.fit(x_train_resampled, y_train_resampled)

y_pred = modelo.predict(x_test_escalonado)

In [24]:
from sklearn.metrics import classification_report, roc_auc_score

auc_score = roc_auc_score(y_test, y_pred)

print(classification_report(y_test, y_pred))

print(scores.mean())

print(auc_score.mean())

              precision    recall  f1-score   support

         0.0       1.00      0.93      0.96    182519
         1.0       0.58      0.95      0.72     17481

    accuracy                           0.93    200000
   macro avg       0.79      0.94      0.84    200000
weighted avg       0.96      0.93      0.94    200000

0.9792708914086447
0.9430106143883958


#Conclusão


O modelo se ajustou bem, mas algumas métricas precisam ser otimizadas

Classe 0 (não fraude): O modelo está quase perfeito em identificar essa classe — precision de 1.00 significa que tudo que ele chamou de "não fraude" realmente não era.

Classe 1 (fraude): Aqui está o desafio. O modelo acerta muitas fraudes (recall = 0.95), mas também erra bastante ao prever fraudes (precision = 0.58) — ou seja, ele gera muitos falsos positivos.